In [ ]:
# 📊 Analytics de Notas con Sentimiento

Análisis de datos de la aplicación de notas usando AWS DynamoDB.

## 🎯 Objetivos:
- Histograma de notas por día
- Gráfico circular de proporción de sentimientos
- Análisis temporal de tendencias emocionales


In [ ]:
# 📦 Instalación de dependencias
!pip install boto3 pandas matplotlib seaborn plotly python-dotenv


In [ ]:
# 🔧 Imports y configuración
import boto3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
from dotenv import load_dotenv
import os

# Configuración de estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de matplotlib
%matplotlib inline

print("✅ Librerías importadas exitosamente")


In [ ]:
# 🔐 Configuración de AWS (ajusta según tu setup)
# Opción 1: Variables de entorno
load_dotenv()

# Configurar credenciales AWS
session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_REGION', 'us-east-1')
)

# Conectar a DynamoDB
dynamodb = session.resource('dynamodb')

# 🗃️ Nombre de tu tabla (ajusta según tu configuración)
TABLE_NAME = 'Note-dev'  # Cambia por el nombre real de tu tabla

try:
    table = dynamodb.Table(TABLE_NAME)
    print(f"✅ Conectado a tabla: {TABLE_NAME}")
    print(f"📊 Status: {table.table_status}")
except Exception as e:
    print(f"❌ Error conectando a DynamoDB: {e}")
    print("💡 Verifica el nombre de la tabla y credenciales AWS")


In [ ]:
# 📥 Obtener datos de DynamoDB
def get_all_notes():
    """Obtiene todas las notas de DynamoDB usando scan"""
    try:
        response = table.scan()
        notes = response['Items']
        
        # Manejar paginación si hay muchas notas
        while 'LastEvaluatedKey' in response:
            response = table.scan(
                ExclusiveStartKey=response['LastEvaluatedKey']
            )
            notes.extend(response['Items'])
        
        print(f"✅ Obtenidas {len(notes)} notas de DynamoDB")
        return notes
    
    except Exception as e:
        print(f"❌ Error obteniendo datos: {e}")
        return []

# Obtener datos
raw_notes = get_all_notes()

if raw_notes:
    print("\n📝 Muestra de datos:")
    for i, note in enumerate(raw_notes[:3]):
        print(f"{i+1}. {note.get('text', 'N/A')[:50]}... - {note.get('sentiment', 'N/A')}")
else:
    print("⚠️ No se encontraron notas. ¿Hay datos en la tabla?")


In [ ]:
# 📈 GRÁFICO 1: Histograma de notas por día
def process_notes(notes):
    """Convierte las notas a DataFrame y procesa fechas"""
    if not notes:
        return pd.DataFrame()
    
    # Crear DataFrame
    df = pd.DataFrame(notes)
    
    # Procesar fechas
    if 'dateCreated' in df.columns:
        df['dateCreated'] = pd.to_datetime(df['dateCreated'])
        df['date'] = df['dateCreated'].dt.date
        df['hour'] = df['dateCreated'].dt.hour
        df['day_of_week'] = df['dateCreated'].dt.day_name()
    
    # Limpiar texto
    if 'text' in df.columns:
        df['text_length'] = df['text'].str.len()
    
    print(f"📊 DataFrame procesado: {len(df)} filas, {len(df.columns)} columnas")
    return df

# Procesar datos
df = process_notes(raw_notes)

if not df.empty and 'date' in df.columns:
    plt.figure(figsize=(12, 6))
    
    # Contar notas por día
    daily_counts = df.groupby('date').size().reset_index(name='count')
    
    # Crear histograma
    plt.subplot(1, 2, 1)
    bars = plt.bar(range(len(daily_counts)), daily_counts['count'], 
                   color='purple', alpha=0.7, edgecolor='black')
    plt.title('📅 Notas Creadas por Día', fontsize=14, fontweight='bold')
    plt.xlabel('Días')
    plt.ylabel('Número de Notas')
    plt.xticks(range(len(daily_counts)), 
               [str(date) for date in daily_counts['date']], 
               rotation=45, ha='right')
    
    # Agregar valores en las barras
    for bar, count in zip(bars, daily_counts['count']):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                 str(count), ha='center', va='bottom')
    
    # Línea de tendencia
    plt.subplot(1, 2, 2)
    plt.plot(daily_counts['date'], daily_counts['count'], 
             marker='o', color='purple', linewidth=2, markersize=8)
    plt.title('📈 Tendencia de Notas en el Tiempo', fontsize=14, fontweight='bold')
    plt.xlabel('Fecha')
    plt.ylabel('Número de Notas')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Total de días con actividad: {len(daily_counts)}")
    print(f"📝 Promedio de notas por día: {daily_counts['count'].mean():.1f}")
    print(f"🏆 Día más productivo: {daily_counts.loc[daily_counts['count'].idxmax(), 'date']} ({daily_counts['count'].max()} notas)")
else:
    print("⚠️ No hay datos suficientes para el histograma de días")


In [ ]:
# 🥧 GRÁFICO 2: Distribución de sentimientos
if not df.empty and 'sentiment' in df.columns:
    plt.figure(figsize=(15, 5))
    
    # Contar sentimientos
    sentiment_counts = df['sentiment'].value_counts()
    
    # Colores para cada sentimiento
    colors = {
        'happy': '#FFD700',    # Dorado
        'sad': '#87CEEB',      # Azul cielo
        'neutral': '#D3D3D3',  # Gris claro
        'angry': '#FF6B6B'     # Rojo coral
    }
    
    # Emojis para cada sentimiento
    emojis = {
        'happy': '😊',
        'sad': '😢', 
        'neutral': '😐',
        'angry': '😠'
    }
    
    # Gráfico circular
    plt.subplot(1, 3, 1)
    wedges, texts, autotexts = plt.pie(sentiment_counts.values, 
                                       labels=[f"{emojis.get(s, s)} {s.title()}" for s in sentiment_counts.index],
                                       colors=[colors.get(s, 'gray') for s in sentiment_counts.index],
                                       autopct='%1.1f%%',
                                       startangle=90,
                                       explode=[0.05] * len(sentiment_counts))
    plt.title('🎭 Distribución de Sentimientos', fontsize=14, fontweight='bold')
    
    # Gráfico de barras
    plt.subplot(1, 3, 2)
    bars = plt.bar(range(len(sentiment_counts)), sentiment_counts.values,
                   color=[colors.get(s, 'gray') for s in sentiment_counts.index],
                   alpha=0.8, edgecolor='black')
    plt.title('📊 Conteo de Sentimientos', fontsize=14, fontweight='bold')
    plt.xlabel('Sentimiento')
    plt.ylabel('Cantidad')
    plt.xticks(range(len(sentiment_counts)), 
               [f"{emojis.get(s, s)}\n{s}" for s in sentiment_counts.index])
    
    # Agregar valores en las barras
    for bar, count in zip(bars, sentiment_counts.values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                 str(count), ha='center', va='bottom', fontweight='bold')
    
    # Tabla de resumen
    plt.subplot(1, 3, 3)
    plt.axis('off')
    
    # Crear tabla
    table_data = []
    for sentiment, count in sentiment_counts.items():
        percentage = (count / len(df)) * 100
        table_data.append([f"{emojis.get(sentiment, sentiment)} {sentiment.title()}", 
                          str(count), f"{percentage:.1f}%"])
    
    table = plt.table(cellText=table_data,
                     colLabels=['Sentimiento', 'Cantidad', 'Porcentaje'],
                     cellLoc='center',
                     loc='center',
                     colColours=['lightblue'] * 3)
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    plt.title('📋 Resumen Estadístico', fontsize=14, fontweight='bold', y=0.8)
    
    plt.tight_layout()
    plt.show()
    
    # Estadísticas
    print("\n📈 Análisis de Sentimientos:")
    print(f"😊 Sentimiento más común: {sentiment_counts.index[0]} ({sentiment_counts.iloc[0]} notas)")
    print(f"😔 Sentimiento menos común: {sentiment_counts.index[-1]} ({sentiment_counts.iloc[-1]} notas)")
    print(f"🎯 Total de notas analizadas: {len(df)}")
else:
    print("⚠️ No hay datos suficientes para análisis de sentimientos")


In [ ]:
# 🎯 RESUMEN EJECUTIVO
if not df.empty:
    print("\n" + "="*60)
    print("📋 RESUMEN EJECUTIVO - ANALYTICS DE NOTAS")
    print("="*60)
    
    print(f"\n📊 DATOS GENERALES:")
    print(f"   📝 Total de notas: {len(df):,}")
    
    if 'dateCreated' in df.columns:
        date_range = df['dateCreated'].max() - df['dateCreated'].min()
        print(f"   📅 Período analizado: {date_range.days + 1} días")
        print(f"   ⚡ Promedio notas/día: {len(df)/(date_range.days + 1):.1f}")
    
    if 'sentiment' in df.columns:
        sentiment_dist = df['sentiment'].value_counts(normalize=True) * 100
        print(f"\n🎭 DISTRIBUCIÓN EMOCIONAL:")
        for sentiment, pct in sentiment_dist.items():
            emoji = {'happy': '😊', 'sad': '😢', 'neutral': '😐', 'angry': '😠'}.get(sentiment, '❓')
            print(f"   {emoji} {sentiment.title()}: {pct:.1f}%")
    
    if 'text_length' in df.columns:
        print(f"\n📝 ANÁLISIS DE TEXTO:")
        print(f"   📏 Longitud promedio: {df['text_length'].mean():.0f} caracteres")
        print(f"   📖 Nota más larga: {df['text_length'].max()} caracteres")
        print(f"   📄 Nota más corta: {df['text_length'].min()} caracteres")
    
    if 'hour' in df.columns:
        peak_hour = df['hour'].value_counts().idxmax()
        print(f"\n⏰ PATRONES TEMPORALES:")
        print(f"   🌟 Hora pico: {peak_hour}:00")
        
        if 'day_of_week' in df.columns:
            peak_day = df['day_of_week'].value_counts().idxmax()
            print(f"   📅 Día más activo: {peak_day}")
    
    print("\n💡 RECOMENDACIONES:")
    
    if 'sentiment' in df.columns:
        positive_ratio = (df['sentiment'] == 'happy').sum() / len(df) * 100
        if positive_ratio > 50:
            print("   ✅ Alto nivel de sentimientos positivos - usuarios satisfechos")
        else:
            print("   ⚠️ Considerar features para mejorar el estado de ánimo")
    
    if len(df) > 0:
        days_with_data = len(df.groupby('date')) if 'date' in df.columns else 1
        engagement = len(df) / days_with_data
        if engagement > 5:
            print("   📈 Excelente engagement - usuarios muy activos")
        elif engagement > 2:
            print("   📊 Buen engagement - usuarios moderadamente activos")
        else:
            print("   📉 Considerar estrategias para aumentar engagement")
    
    print("\n🎯 PRÓXIMOS PASOS:")
    print("   1. Implementar notificaciones en horas pico")
    print("   2. Agregar análisis de tendencias semanales")
    print("   3. Crear dashboard en tiempo real")
    print("   4. Implementar ML para predicción de sentimientos")
    
    print("\n" + "="*60)
else:
    print("\n❌ No hay datos suficientes para generar un resumen ejecutivo")
    print("💡 Asegúrate de:")
    print("   1. Tener notas creadas en tu aplicación")
    print("   2. Verificar la conexión a DynamoDB")
    print("   3. Confirmar el nombre de la tabla")


In [ ]:
## 🚀 Cómo usar este notebook

### 📋 Preparación:
1. **Configura tus credenciales AWS** en un archivo `.env`:
   ```
   AWS_ACCESS_KEY_ID=tu_access_key
   AWS_SECRET_ACCESS_KEY=tu_secret_key
   AWS_REGION=us-east-1
   ```

2. **Ajusta el nombre de la tabla** en la celda de configuración según tu setup de DynamoDB

3. **Instala las dependencias** ejecutando la primera celda

### 🎯 Funcionalidades:
- ✅ **Histograma por días** - Visualiza la actividad diaria
- ✅ **Gráfico circular** - Distribución de sentimientos
- ✅ **Análisis temporal** - Patrones por hora y día de la semana
- ✅ **Resumen ejecutivo** - Insights y recomendaciones

### 💡 Tips:
- Ejecuta todas las celdas en orden
- Si no tienes datos, crea algunas notas primero en tu app
- Los gráficos se adaptan automáticamente al volumen de datos
- Puedes personalizar colores y estilos en el código

---
*Notebook creado para el coding challenge - Analytics de Notas con Sentimiento* 💜
